# Waterpoint Data Prep

In [ ]:
#Brian, if you don't have these packages, run this cell
!pip install fuzzywuzzy
!pip install pandas
!pip install numpy

## Update the "INPUT_FILE" Path below with your local file location of the complete WPDX data set 

## The first half cleans up the columns and adds some data for the age of the well and time since measurement

## The second half runs fuzzy matching. The taxonomy is contained in two places. These were obtained from exploratory analysis and some trial and error

### Taxonomy for Water Tech
choices_tech=
['Borehole', 'India Mark MK IMK II', 'Afridev', 'Gravity', 'Hand Pump','Electrical Pump', 'Bush Pump', 
          'Tube Well', 'Standpipe Stand Post', 'tap', 'Bucket','Indus', 'Pamir', 'Kardia', 'Kabul KABUL',
           'Spring River Stream Protected','Lake Pond Dam', 'municipal water supply', 'Rain Rainwater', 'Tank', 
          '__MISSING__', 'Well']

### Taxonomy for Water Sources.
source_choices= ['Borehole', 'Tube Well', 'Well', 'Stand Post Tap','Hand Pump', 'Piped', 'Spring River Stream Protected', 'Lake Pond Dam', 'Spring River Stream Unprotected', 'Sand','municipal water supply',  'Rain Rainwater', 'Tank',  '__MISSING__','Unknown']  

In [24]:
#set display options
pd.set_option('max_columns', 150)
pd.set_option('max_rows', 500)


In [16]:
#IMPORTS
import pandas as pd
import numpy as np
import os
import fuzzywuzzy
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [3]:

#set input/output paths
#can eventually set this to the SOAPY API https://dev.socrata.com/foundry/data.waterpointdata.org/gihr-buz6
DATA_PATH = "/Users/chandler.mccann/Downloads"
INPUT_FILE = os.path.join(DATA_PATH, "Water_Point_Data_Exchange_Complete_Dataset.csv")
OUTPUT_FILE = os.path.join(DATA_PATH, "cleaned_water_data2.csv")

print("reading in input file from", INPUT_FILE)
#read in the file
df = pd.read_csv(INPUT_FILE, encoding='latin-1')

print("cleaning headings and creating features")
def clean_headings(df):
    #clean column names
    df.columns = [c.replace('#', '') for c in df.columns]
    return df

df=clean_headings(df)    

#convert date and year columns to datetime
df['new_report_date'] = pd.to_datetime(df.report_date)
df['new_install_year'] = pd.to_datetime(df.install_year,format='%Y.0')

#make age of well feature.  This will return in days, which assumes install year starts on Jan 1.  We may want to 
#make more date features.
# also tree methods may have a better time with 3.2 years than 1018 days
# this returns 0 days and NaTs.  Fill NaT with 9999 in XGB

df['age_well']=df.new_report_date - df.new_install_year

#ugly work around buy age_well comes out with hours, min and UTC.  Converting to string, splitting, and taking the days digits
df['age_well_days']=df.age_well.apply(lambda x: str(x).split()[0])

#binary target.  When status is yes == 0, when no OR maybe == 1
df['status_binary']=np.where(df.status_id=='yes',0,1)

#make a variable for time since last measurement
df['time_since_measurement'] = pd.to_datetime('today') - pd.to_datetime(df.new_report_date)

#convert it to years. It's a timedelta time stamp
df['time_since_meas_years']=df.time_since_measurement.apply(lambda x: round(x.days/365,1))

#Obscure missing values in water tech need to map: 0," ", other ,etc to __MISSING__ so that we have consistent values for
#__MISSING__
df.water_tech.replace({"0": "__MISSING__", " ": "__MISSING__","Other": "__MISSING__","None Other": "__MISSING__", "Unidentified":"__MISSING__" }, inplace=True)


def fill_missing(df):
    """args: pandas data frame
       returns: a pandas data frame that converted age_well_days and fills missing values """
    
    if 'age_well_days' in df.columns:
        df.replace(to_replace='NaT', value=99999, inplace=True) #have to replace the NaTs. Using a long value for missing years so tree picks up
        #convert to int since we dont want to label encode this
        df['age_well_days']= df['age_well_days'].astype(str).astype(int)
        
    #get columns that are of type object
    cols = df.select_dtypes(include=['object']).columns
    
    #fill with __MISSING___
    for col in cols:
        df[col].fillna('__MISSING__', inplace=True)
    
    #new_install_year not filling? hardcoding in now but should fix.  Something to do with forcing pd.Datetime to Y%?
    for annoying_col in ['install_year', 'new_install_year']:
        df[annoying_col].fillna('__MISSING__', inplace=True)
      
    return df


def make_well_years(df):
    if 'age_well_days' in df.columns and df.age_well_days.isnull().sum()==0:
        df['age_well_years'] = round(df.age_well_days/365,1)
    else:
        pass
    return df
    
#Apply fuzzy matching given a set of choices and a scorer
#Below we define choices for the "Water Tech" column--> These are in "choices_tech"
#We also define choices for the "Water Source" collumn--> These are in "sources_choices"
def fuzzymatch2(x, choices, scorer, cutoff):
    results = process.extractOne(
    x, choices=choices, scorer=scorer, score_cutoff= cutoff)
    if results is None:
        return "no match"
    else:
        return results[0] #only return the match word of the tuple, exclude the score

#define fuzzy matching choices for water tech
choices_tech= ['Borehole', 'India Mark MK IMK II', 'Afridev', 'Gravity', 'Hand Pump','Electrical Pump', 'Bush Pump', 
          'Tube Well', 'Standpipe Stand Post', 'tap', 'Bucket','Indus', 'Pamir', 'Kardia', 'Kabul KABUL',
           'Spring River Stream Protected','Lake Pond Dam', 'municipal water supply', 'Rain Rainwater', 'Tank', 
          '__MISSING__', 'Well']
          
source_choices= ['Borehole', 'Tube Well', 'Well', 'Stand Post Tap','Hand Pump', 'Piped', 'Spring River Stream Protected', 
                 'Lake Pond Dam', 'Spring River Stream Unprotected',
                 'Sand','municipal water supply',  'Rain Rainwater', 'Tank',  '__MISSING__ Unknown']          

print("filling missing values....")         
#run the functions
df= fill_missing(df)

df = make_well_years(df)

print("executing fuzzy matching...")

"""Execute fuzzy matching on water sources.  Returns "no match" if the fuzzy score is 60 . The "Token Set Ratio" metric
is used.  Below are the metric options to match. We settled on Token Set Ratio due to the nature of the data (common words that were
shuffled out of order, capitalized or not)

Simple Ratio - Pure Levenshtein Distance based matching
Partial Ratio - Matches based on best substrings
Token Sort Ratio - Tokenises strings and sorts them alphabetically before matching
IN USE -->Token Set Ratio - Tokenise and compare intersection and remainder"""

df['fuzzy_water_source']= df.loc[:,'water_source'].apply(
                                            fuzzymatch2,
                                            args=(source_choices, 
                                                  fuzz.token_set_ratio, 
                                                 60)
)

df['fuzzy_water_tech']= df.loc[:,'water_tech'].apply(
                                            fuzzymatch2,
                                            args=(choices_tech, 
                                                  fuzz.token_set_ratio, 
                                                 60)
)
#drop row ID
df.drop(['Row ID'], axis=1, inplace = True)

#drop duplicates
df.drop_duplicates(inplace=True)

print("writing to csv to ", OUTPUT_FILE)
#write to csv
df.to_csv(OUTPUT_FILE, index=False)

print("DONE!")

reading in input file from /Users/chandler.mccann/Downloads/Water_Point_Data_Exchange_Complete_Dataset.csv


/Users/chandler.mccann/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,6,8,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


cleaning headings and creating features
filling missing values....
executing fuzzy matching...
writing to csv to  /Users/chandler.mccann/Downloads/cleaned_water_data2.csv
DONE!


In [5]:
df= pd.read_csv('cleaned_water_data2.csv')

/Users/chandler.mccann/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (16,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df.columns

Index(['country_name', 'water_source', 'water_tech', 'status_id',
       'status_0_yes', 'management', 'pay', 'installer', 'install_year',
       'status', 'source', 'adm1', 'adm2', 'wpdx_id', 'report_date',
       'country_id', 'activity_id', 'data_lnk', 'orig_lnk', 'photo_lnk',
       'converted', 'created', 'updated', 'lat_deg', 'lon_deg', 'Location',
       'Count', 'fecal_coliform_presence', 'fecal_coliform_value',
       'subjective_quality', 'new_report_date', 'new_install_year', 'age_well',
       'age_well_days', 'status_binary', 'time_since_measurement',
       'time_since_meas_years', 'age_well_years', 'fuzzy_water_source',
       'fuzzy_water_tech'],
      dtype='object')

In [12]:
print("total num water sources", len(df.water_source.unique()))
print("total num water tech", len(df.water_tech.unique()))
print("total num matched water sources", len(df.fuzzy_water_source.unique()))
print("total num matched water tech", len(df.fuzzy_water_tech.unique()))

total num water sources 378
total num water tech 1159
total num matched water sources 16
total num matched water tech 24


In [27]:
#look at what is match
pd.DataFrame(df.groupby(['fuzzy_water_tech', 'water_tech']).size())

0
fuzzy_water_tech water_tech                                               
Afridev          AfriDev Handpump                                     2017
                 Afridef                                                 1
                 Afridev                                             13271
                 Afridev Cattle trough                                   3
                 Afridev Communal standpipe                             15
                 Afridev Communal standpipe multiple                    15
                 Afridev Hand pump                                     701
                 Afridev Not recorded                                    5
                 Afridev Other                                           4
                 Afridev Pump                                            5
                 Dugwell Afridev                                        11
                 Electrical (surface)-Afridev                            6
                 Electrical Submersible-Afridev                          6
                 Hand Pump-Afridev                                      36
                 Hand Pump|Afridev                                    1139
                 Hand Pump|Afridev|Diesel                                6
                 Hand Pump|Afridev|Electricity                           1
                 Hand pump|Afridev                                     123
                 Manual/foot-Afridev                                   956
                 Solar Driven Pump-Afridev                               1
                 Submersible pump|Afridev                                1
                 Treadle Pump-Afridev                                    1
                 afridev                                                 1
                 |Afridev                                                5
Borehole         BORE HOLE                                               1
                 BOREHOLE                                                2
                 Borehole                                             1469
                 Borehole or tubewell                                   47
                 Borehole with hand pump                               174
                 Borehole with hand pump (Afridev)                      56
                 Borehole with hand pump (India Mark II / III)           1
                 Borehole with hand pump|Borehole with foot pump         2
                 Borehole with hand pump|Borehole with wind pump         1
                 Borehole with hand pump|Solar power                     2
                 DEEP BORE HOLE                                          1
                 DEEP BOREHOLE                                           1
                 Deep borehole                                       25959
                 Mechanized Borehole                                   272
                 Mechanized Borehole-Generator                          29
                 Mechanized Borehole-Grid                               43
                 Mechanized Borehole-Solar                              43
                 Mechanized Borehole-Wind                                3
                 Non-Mechanized Borehole                              3893
                 Non-Mechanized Borehole-Solar                           4
                 Others|Other|The borehole is capped|                    1
                 Pipeline extension from borehole to dispensary ...      1
                 Pipeline extension from borehole to school and ...      1
                 Protected Deep Borehole                               128
                 Protected Deep Borehole - Gravity Fed System            4
                 Protected Deep Borehole - Pumped piped system (...     19
                 Protected Deep Borehole - Pumped piped system (...      2
                 Shallow Borehole                                       23
Bucket           A smal

In [26]:
#look at what is match
pd.DataFrame(df.groupby(['fuzzy_water_source', 'water_source']).size())

0
fuzzy_water_source              water_source                                              
Borehole                        Borehole                                             19556
                                Borehole Pump To Tap Water                               1
                                Borehole fitted with Manual pump                         2
                                Borehole fitted with manual pump                       799
                                Borehole fitted with no pump                            19
                                Borehole fitted with submersible pump                 9997
                                Boreholes                                             1874
                                Existing Borehole                                        2
                                Machine Drilled Borehole                               854
                                Machine-drilled borehole                              3789
                                Manual pump on borehole                               1257
                                Manually drilled borehole                               26
                                Mechically drilled borehole                            198
                                New Borehole                                            19
                                OTHER:Bore hole                                          1
                                OTHER:Borehole                                           2
                                OTHER:Borehole with uplift pump                          1
                                Protected Borehole                                   16828
                                Protected Deep Borehole                                766
                                Pump on borehole                                      1935
                                Tap Water From Macalder Borehole                         1
                                Tube well or borehole                                  997
                                Tubewell or borehole                                   150
                                Unequiped borehole                                       2
                                Unequipped borehole                                     72
                                Unequipped borehole/capped                               9
                                Unprotected Borehole                                  2880
                                borehole                                              1602
                                borehole with troughs                                    2
                                hand drilled borehole                                    2
                                mechanic borehole                                      403
Hand Pump                       Automatic pump                                           3
                                Dam connected to a pump                                  1
                                Hand Pumps                                            1964
                                OTHER:Colonial pump                                      1
                                OTHER:Hand pump                                         51
                                OTHER:Handpump                                          17
                                OTHER:Jake pump                                          2
                                OTHER:Protected hand dug with submerssible pump          1
                                OTHER:Pump removed water drown mannually                 1
                                OTHER:Submasive pump                                     1
                                OTHER:Tap Pump                                           3
                                OTHER:banda pump                                     

In [29]:
#look at what we couldn't match
df.loc[df.fuzzy_water_tech =='no match'].groupby('water_tech').size().sort_values(ascending=False)

water_tech
Unprotected  Birka                                             1454
Kiosk with Piped Supply                                        1303
Storage Facility                                                968
Kiosk                                                           800
Inkar                                                           615
Pompe manuelle                                                  511
Consallen                                                       393
PHED Piped Supply System                                        382
BPT                                                             349
Pompe ÌÊ motricitÌ© humaine                                     291
No Handpump Installed                                           280
Rowa and Joma                                                   267
Mono Cattle trough                                              251
Protected Point source- connected to piped scheme               244
Wind Mill                            

In [30]:
df.loc[df.fuzzy_water_source =='no match'].groupby('water_tech').size().sort_values(ascending=False)

water_tech
__MISSING__                                      1865
no pump                                          1025
Gravity Not recorded                              540
Pompe manuelle                                    511
Standpipe                                         434
INDUS                                             270
Hand / manual (e.g. rope pump, rope & bucket)     179
KABUL                                             118
Gravity Other                                      96
nira                                               81
india_mk_ii                                        81
PAMIR                                              70
Gravity                                            66
ferro_cement_tank                                  47
Other (please specify)                             47
Bucket Only                                        41
Puisettes                                          30
None Not recorded                                  29
IMK II           

In [31]:
pd.DataFrame(df.groupby(['fuzzy_water_tech','water_tech']).size()).to_csv('fuzzy_water_tech_results.csv')

In [32]:
pd.DataFrame(df.groupby(['fuzzy_water_source','water_source']).size()).to_csv('fuzzy_water_source_results.csv')